In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display, HTML

from sklearn.datasets import fetch_20newsgroups 

newsgroup = fetch_20newsgroups()

# les messages
messages = newsgroup['data']

# répartition des messages
solution = pd.Series(newsgroup['target'])
group_df = pd.DataFrame(solution.value_counts().sort_index(), columns=['msg_count'])
group_df.insert(0, 'group_names', newsgroup['target_names'])

print(group_df)
print(len(messages))

                 group_names  msg_count
0                alt.atheism        480
1              comp.graphics        584
2    comp.os.ms-windows.misc        591
3   comp.sys.ibm.pc.hardware        590
4      comp.sys.mac.hardware        578
5             comp.windows.x        593
6               misc.forsale        585
7                  rec.autos        594
8            rec.motorcycles        598
9         rec.sport.baseball        597
10          rec.sport.hockey        600
11                 sci.crypt        595
12           sci.electronics        591
13                   sci.med        594
14                 sci.space        593
15    soc.religion.christian        599
16        talk.politics.guns        546
17     talk.politics.mideast        564
18        talk.politics.misc        465
19        talk.religion.misc        377
11314


In [25]:
# fonction qui va séparer les données de chaque message
def format_message(str) -> dict['headers':dict, 'message':str]: 

    lines = str.split('\n')

# 1. GET HEADERS
    headers = {}
    message_index = 0
    last_header = ''
    for i,l in enumerate(lines) :
        # Détetcter le premier saut de ligne qui represente le début du message
        if l == '' :
            message_index = i
            break
        
        head = l.split(':',1)
        # gestion des exceptions
        if len(head) < 2 :
            # aucune entête
            if last_header == '' :
                if 'TRASH' not in headers :  headers['TRASH'] = ''
                headers['TRASH'] += (head[0])
            # entête multi-ligne
            else : 
                headers[last_header] += (head[0])
        # cas normal
        else :
            last_header = head[0]
            headers[head[0]] = head[1][1:]

# 2. GET MESSAGE
    message = '\n'.join(lines[message_index:])

    return {'headers': headers, 'message': message}

In [26]:
for k,v in format_message(messages[690])['headers'].items():
    print(k,':',v)

TRASH :  cs.utexas.edu!uunet!olivea!sgigate!sgiblab!adagio.panasonic.com!nntp-server.caltech.edu!keith
Subject : Re: <Political Atheists?
From : keith@cco.caltech.edu (Keith Allan Schneider) <930401.112329.0u1.rusnews.w165w@mantis.co.uk> <11710@vice.ICO.TEK.COM>
Organization : California Institute of Technology, Pasadena
NNTP-Posting-Host : lloyd.caltech.edu
Lines : 17


In [27]:
from pprint import pprint

# Faire la liste de tous les headers pour voir haha
headers_list = {}

for i, msg in enumerate(messages):
    headers = format_message(msg)['headers']
    for header in headers.keys():
        if header not in headers_list:
            headers_list[header] = 1
        else:
            headers_list[header] += 1

print(len(headers_list))

# filtrage des entêtes les plus présentes
limite = 999
main_head = dict(filter(lambda a: True if a[1] > limite else False, headers_list.items()))

pprint(main_head)


119
{'Distribution': 2533,
 'From': 11314,
 'Lines': 11276,
 'NNTP-Posting-Host': 2311,
 'Nntp-Posting-Host': 2453,
 'Organization': 10841,
 'Reply-To': 1720,
 'Subject': 11314}


In [28]:
# créer un dataframe qui va contenir ces headers pour aider à catégoriser les messages
# laisser le texte et la signature en brut

# liste des messages
mess_list = []
# dict des headers, chaque valeur est une liste de la longueur du tableau
head_dict = {}
for h in main_head.keys():
    head_dict[h] = []

# remplissage de ces objets
for msg in messages:
    mf = format_message(msg)
    # garniture des messages
    mess_list.append(mf['message'][1:])
    # garniture des headers
    to_fill = list(head_dict.keys()) # réduis à mesure que les valeurs sont renseignées
    for header, value in mf['headers'].items():
        if header in head_dict:
            head_dict[header].append(value)
            to_fill.remove(header)
    for h in to_fill: head_dict[h].append(None) # remplis les valeurs non renseignées

df = pd.DataFrame.from_dict(head_dict)
df['Message'] = mess_list

# export vers un format de stockage (le csv pose un problème, je ne sais pas quel séparateur utiliser)
df.to_json('./newsgroup.json')

Je crois que je comprends ce qu’il faut faire : 
(c’est faux)
1. faire une classification des textes du subset train avec un algo genre kmeans (pas retenu les autres)\
Donc on a pas besoin de faire un trainTestSplit en fait ?\
Ou alors on travaille avec toutes les données et on split nous même, pourquoi pas.\
Mais la grande question c’est si on peut fournir le Y ?
On fait de l’apprentissage supervisé ou non ?
2. vectoriser chaque groupe obtenu. Idéalement avec les deux methodes : CountVectorizer et TfIDFVectorizer\
Question : Comment les attribuer à un groupe ? 
3. diviser les textes du groupe de test selon leurs catégories (avec subset='test')
4. les vectoriser aussi ?
5. comparer les deux (je vois pas comment)

In [38]:
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans

# copier le code de test de kmeans

def bench_k_means(estimator, data, y=None):
    estimator.fit(data)
    scores = [
        "adjusted_rand_score",
        "adjusted_mutual_info_score",
        "silhouette_score",
        "homogeneity_score",
        "completeness_score",
        "v_measure_score",
    ]

    results = []
    for s in scores:
        if s not in ["silhouette_score"]:
            # with ground-truth
            # Yeah !
            score = getattr(metrics, s)(y, estimator.labels_)
        elif y is not None:
            # with NO ground-truth
            score = getattr(metrics, s)(data, estimator.labels_)
        else:
            continue
            
        results.append(score)
        
    return pd.DataFrame([results], columns=scores, index=["result"])

In [95]:
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
print(len(stops))
print('---- (❁´◡`❁) ----')
# C’est pire que l’autre stopwords ??

# les données
Xtrain, Ytrain = fetch_20newsgroups(return_X_y=True, remove=('headers', 'footers', 'quotes'))
groups = fetch_20newsgroups()['target_names']

# Bien sur les phrases doivent être vectorisées
c_vec = CountVectorizer(max_features=20, stop_words='english', token_pattern=r'(?u)\b[a-zA-Z]\w+\b')
X_vec = c_vec.fit_transform(Xtrain[:20])
feat_n = c_vec.get_feature_names_out()
print(feat_n)

# autre methode de vectorisation
t_vec = TfidfVectorizer(max_features=20, stop_words='english', token_pattern=r'(?u)\b[a-zA-Z]\w+\b')
X_vec = t_vec.fit_transform(Xtrain[:20])
feat_n = c_vec.get_feature_names_out()
print(feat_n)
# print(X_vec.toarray())

179
---- (❁´◡`❁) ----
['board' 'capability' 'car' 'does' 'don' 'flights' 'info' 'insurance'
 'know' 'new' 'option' 'parent' 'power' 'rate' 'scsi' 'ssf' 'tiff' 've'
 'year' 'years']
['board' 'capability' 'car' 'does' 'don' 'flights' 'info' 'insurance'
 'know' 'new' 'option' 'parent' 'power' 'rate' 'scsi' 'ssf' 'tiff' 've'
 'year' 'years']


In [92]:
vectorizer = TfidfVectorizer(
    max_features=5000, 
    min_df=0.01, 
    max_df=0.8, 
    stop_words='english',
    token_pattern=r'(?u)\b[a-zA-Z]\w+\b'
    )
X_vector = vectorizer.fit_transform(Xtrain)

model = KMeans(n_clusters=20, n_init='auto')
model.fit(X_vector)
x_trans = model.transform(X_vector)
x_categorized = model.predict(X_vector)

print(len(x_categorized))


11314


In [93]:
main_words_by_category = {}

for i,cluster in enumerate(model.cluster_centers_):
    main_words = cluster.argsort()[-10:]
    main_words_by_category[f'Cluster{i}'] = []
    for word in main_words:
        main_words_by_category[f'Cluster{i}'].append(vectorizer.get_feature_names_out()[word])
    
pprint(main_words_by_category)

{'Cluster0': ['time',
              'don',
              'god',
              'know',
              'believe',
              'just',
              'people',
              'christian',
              'church',
              'group'],
 'Cluster1': ['right',
              'good',
              'time',
              'really',
              'say',
              'did',
              'like',
              'know',
              'think',
              'don'],
 'Cluster10': ['anybody',
               'looking',
               'info',
               'help',
               'hi',
               'does',
               'mail',
               'know',
               'advance',
               'thanks'],
 'Cluster11': ['problem',
               'pub',
               'use',
               'windows',
               'program',
               'ftp',
               'format',
               'directory',
               'files',
               'file'],
 'Cluster12': ['don',
               'mit',
               'j

In [94]:
# compter le nombre de vraies catégories dans chaque cluster
# print(len(x_categorized), len(Ytrain), len(Xtrain))

df_cluster = pd.DataFrame(x_categorized, columns=['Cluster'])
df_cluster['Target'] = Ytrain

df_cluster = df_cluster.groupby(['Cluster', 'Target']).size()

print(df_cluster)

target_pop = group_df['msg_count'].to_list()
print(target_pop)

for i in range(16):
    print(f'-- CLUSTER {i} --')
    pop_dict = df_cluster[i].to_dict()
    print(pop_dict)
    for k,v in pop_dict.items():
        ratio = v/target_pop[int(k)]
        if ratio > 0.2:
            msg = f'{int(ratio*100)}% du groupe {k}'
            print(msg)


Cluster  Target
0        0         12
         1         12
         2          7
         3          2
         4          2
                   ..
15       8          7
         9          2
         12         8
         14         1
         18         2
Length: 255, dtype: int64
[480, 584, 591, 590, 578, 593, 585, 594, 598, 597, 600, 595, 591, 594, 593, 599, 546, 564, 465, 377]
-- CLUSTER 0 --
{0: 12, 1: 12, 2: 7, 3: 2, 4: 2, 5: 10, 7: 7, 8: 8, 9: 1, 10: 3, 11: 7, 12: 4, 13: 14, 14: 3, 15: 76, 16: 9, 17: 6, 18: 4, 19: 17}
-- CLUSTER 1 --
{0: 133, 1: 42, 2: 17, 3: 23, 4: 44, 5: 23, 6: 14, 7: 80, 8: 151, 9: 75, 10: 55, 11: 68, 12: 63, 13: 175, 14: 100, 15: 101, 16: 122, 17: 64, 18: 103, 19: 91}
27% du groupe 0
25% du groupe 8
29% du groupe 13
22% du groupe 16
22% du groupe 18
24% du groupe 19
-- CLUSTER 2 --
{3: 1, 4: 1, 5: 1, 6: 8, 8: 3, 9: 233, 10: 317, 12: 1, 14: 3, 17: 1, 18: 1, 19: 1}
39% du groupe 9
52% du groupe 10
-- CLUSTER 3 --
{0: 141, 1: 195, 2: 129, 3: 148, 4: 200, 5: 19